<a href="https://colab.research.google.com/github/kkettip/datasci_2_manipulation/blob/main/datasci_2_manipulation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Importing Libraries

In [1]:
!pip install pandas==2.0
!pip install modin==0.23.0
!pip install brotli
import brotli

from distributed import Client
client = Client()


import modin.pandas as mpd
import pandas as pd
import numpy as np
import re
import polars as pl
import time

INFO:distributed.http.proxy:To route to workers diagnostics web server please install jupyter-server-proxy: python -m pip install jupyter-server-proxy
INFO:distributed.scheduler:State start
INFO:distributed.scheduler:  Scheduler at:     tcp://127.0.0.1:46657
INFO:distributed.scheduler:  dashboard at:  http://127.0.0.1:8787/status
INFO:distributed.nanny:        Start Nanny at: 'tcp://127.0.0.1:43985'
INFO:distributed.nanny:        Start Nanny at: 'tcp://127.0.0.1:46279'
INFO:distributed.scheduler:Register worker <WorkerState 'tcp://127.0.0.1:45047', name: 0, status: init, memory: 0, processing: 0>
INFO:distributed.scheduler:Starting worker compute stream, tcp://127.0.0.1:45047
INFO:distributed.core:Starting established connection to tcp://127.0.0.1:48012
INFO:distributed.scheduler:Register worker <WorkerState 'tcp://127.0.0.1:45657', name: 1, status: init, memory: 0, processing: 0>
INFO:distributed.scheduler:Starting worker compute stream, tcp://127.0.0.1:45657
INFO:distributed.core:Sta

# Loading dataset

In [2]:

data = pd.read_csv('/content/healthcare_data_cleaning.csv')
data

,Patient Age,Gender,City of Residence,State of Residence,Has Insurance,Visited Last Month,Payment Method,Preferred Doctor,Disease Diagnosed,Medication Prescribed,Type of Appointment,Average Heart Rate,Average BP,Height (in cm),Weight (in kg),Payment Due ($),Last Visit (days ago),Visit Duration (mins),Number of Tests,Prescription Cost ($)
0,45,Other,West Brian,Kentucky,Yes,Yes,Insurance,Dr. Williams,Flu,Med_E,General,66,111,183,70,65.45721578126224,193,92,5,15.71751735532889
1,25,Female,East Jocelynfurt,Colorado,No,Yes,Card,Dr. Johnson,Flu,Med_B,Specialist,59,missing,174,58,430.68365678679174,195,missing,4,80.78647284463952
2,51,Other,South Lindseyland,Ohio,Yes,No,Cash,Dr. Williams,Covid-19,Med_D,General,79,119,161,56,315.0709305262176,missing,37,2,64.3921393009105
3,18,Female,Taylorfort,Pennsylvania,Yes,missing,Cash,Dr. Williams,Flu,Med_A,General,99,115,171,52,320.2998987723972,175,108,2,8.872859215315316
4,48,Female,Birdmouth,Montana,No,No,Insurance,Dr. Brown,NaN,Med_C,Specialist,93,89,186,87,264.2147372473319,188,113,3,77.48311304645532
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
104995,74,Other,West Anthony,Oklahoma,Yes,No,Card,Dr. Williams,Covid-19,Med_E,Specialist,65,112,169,61,309.16444606680665,183,10,3,63.20478776719894
104996,37,Male,Mckeetown,Utah,Yes,No,Cash,Dr. Jones,Covid-19,Med_C,General,missing,111,165,67,missing,191,47,2,113.86764025370326
104997,51,Female,Lake Lisachester,Nebraska,Yes,Yes,Insurance,Dr. Williams,Allergy,Med_D,Follow-Up,89,118,179,71,320.8304180033907,172,39,3,35.622420232475356
104998,missing,Female,Heidiborough,missing,Yes,No,Cash,Dr. Smith,Covid-19,Med_A,Follow-Up,74,123,167,75,5.341231420036174,180,79,missing,124.4083409893742


# Obtaining List of Column Names

In [3]:
list(data)

['Patient Age',
 'Gender',
 'City of Residence',
 'State of Residence',
 'Has Insurance',
 'Visited Last Month',
 'Payment Method',
 'Preferred Doctor',
 'Disease Diagnosed',
 'Medication Prescribed',
 'Type of Appointment',
 'Average Heart Rate',
 'Average BP',
 'Height (in cm)',
 'Weight (in kg)',
 'Payment Due ($)',
 'Last Visit (days ago)',
 'Visit Duration (mins)',
 'Number of Tests',
 'Prescription Cost ($)']

# Analyzing Shape of Dataset

In [4]:
print("Number of rows:", data.shape[0])
print("Number of columns:", data.shape[1])
print("Size:", data.size)

Number of rows: 105000
Number of columns: 20
Size: 2100000


# **1. Data Cleaning with Pandas**

# Clean column names
###Removing Leading and Trailing White Space and Replacing Space with Underscore

In [5]:
data.columns = data.columns.str.strip().str.replace(' ', '_')
data

,Patient_Age,Gender,City_of_Residence,State_of_Residence,Has_Insurance,Visited_Last_Month,Payment_Method,Preferred_Doctor,Disease_Diagnosed,Medication_Prescribed,Type_of_Appointment,Average_Heart_Rate,Average_BP,Height_(in_cm),Weight_(in_kg),Payment_Due_($),Last_Visit_(days_ago),Visit_Duration_(mins),Number_of_Tests,Prescription_Cost_($)
0,45,Other,West Brian,Kentucky,Yes,Yes,Insurance,Dr. Williams,Flu,Med_E,General,66,111,183,70,65.45721578126224,193,92,5,15.71751735532889
1,25,Female,East Jocelynfurt,Colorado,No,Yes,Card,Dr. Johnson,Flu,Med_B,Specialist,59,missing,174,58,430.68365678679174,195,missing,4,80.78647284463952
2,51,Other,South Lindseyland,Ohio,Yes,No,Cash,Dr. Williams,Covid-19,Med_D,General,79,119,161,56,315.0709305262176,missing,37,2,64.3921393009105
3,18,Female,Taylorfort,Pennsylvania,Yes,missing,Cash,Dr. Williams,Flu,Med_A,General,99,115,171,52,320.2998987723972,175,108,2,8.872859215315316
4,48,Female,Birdmouth,Montana,No,No,Insurance,Dr. Brown,NaN,Med_C,Specialist,93,89,186,87,264.2147372473319,188,113,3,77.48311304645532
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
104995,74,Other,West Anthony,Oklahoma,Yes,No,Card,Dr. Williams,Covid-19,Med_E,Specialist,65,112,169,61,309.16444606680665,183,10,3,63.20478776719894
104996,37,Male,Mckeetown,Utah,Yes,No,Cash,Dr. Jones,Covid-19,Med_C,General,missing,111,165,67,missing,191,47,2,113.86764025370326
104997,51,Female,Lake Lisachester,Nebraska,Yes,Yes,Insurance,Dr. Williams,Allergy,Med_D,Follow-Up,89,118,179,71,320.8304180033907,172,39,3,35.622420232475356
104998,missing,Female,Heidiborough,missing,Yes,No,Cash,Dr. Smith,Covid-19,Med_A,Follow-Up,74,123,167,75,5.341231420036174,180,79,missing,124.4083409893742


In [6]:
data.columns

Index(['Patient_Age', 'Gender', 'City_of_Residence', 'State_of_Residence',
       'Has_Insurance', 'Visited_Last_Month', 'Payment_Method',
       'Preferred_Doctor', 'Disease_Diagnosed', 'Medication_Prescribed',
       'Type_of_Appointment', 'Average_Heart_Rate', 'Average_BP',
       'Height_(in_cm)', 'Weight_(in_kg)', 'Payment_Due_($)',
       'Last_Visit_(days_ago)', 'Visit_Duration_(mins)', 'Number_of_Tests',
       'Prescription_Cost_($)'],
      dtype='object')

# **Identify and handle missing values in the dataset.**


In [7]:
data.sample(10)


,Patient_Age,Gender,City_of_Residence,State_of_Residence,Has_Insurance,Visited_Last_Month,Payment_Method,Preferred_Doctor,Disease_Diagnosed,Medication_Prescribed,Type_of_Appointment,Average_Heart_Rate,Average_BP,Height_(in_cm),Weight_(in_kg),Payment_Due_($),Last_Visit_(days_ago),Visit_Duration_(mins),Number_of_Tests,Prescription_Cost_($)
7451,37,Male,Jerryside,Kentucky,missing,No,Card,Dr. Smith,Flu,Med_E,Emergency,83,104,185,99,158.9691748714015,184,73,1,121.9224398098515
28148,48,Other,Austinfort,Missouri,No,Yes,Card,Dr. Brown,Covid-19,Med_A,Emergency,82,132,189,81,201.8337032307701,178,65,2,124.16248975008835
61434,65,Female,West Thomasfort,Rhode Island,No,No,Cash,Dr. Smith,NaN,Med_C,Emergency,68,102,missing,77,92.02857608826132,176,70,4,84.69186484756507
22453,23,Male,West Tommy,Wisconsin,No,Yes,Card,Dr. Johnson,missing,Med_C,Specialist,97,80,156,84,54.99487686154225,179,68,6,91.60524734653256
49159,66,Female,Gregoryborough,Montana,No,Yes,Insurance,Dr. Jones,Flu,Med_B,Emergency,76,100,174,57,60.870700363438445,173,39,6,92.9063203293869
17142,30,Other,Larsonfort,West Virginia,No,No,Card,Dr. Smith,Allergy,Med_E,General,missing,99,163,52,330.4917662501116,181,missing,5,114.33093331453158
67269,66,Male,Michaelside,Texas,Yes,Yes,Insurance,Dr. Jones,Allergy,Med_A,General,78,100,178,68,208.665623603758,190,79,2,118.78666644234869
25034,52,Other,Lake Michael,New Hampshire,Yes,Yes,Insurance,Dr. Williams,Allergy,Med_C,Emergency,85,103,190,43,43.82507966594507,176,75,5,132.5387242444056
85305,42,Other,Santiagofurt,Nebraska,Yes,No,Insurance,Dr. Brown,NaN,Med_B,Specialist,88,80,missing,66,121.15919412483727,196,99,2,11.989657217618783
52051,31,Female,New Alexandra,Indiana,No,Yes,Cash,Dr. Smith,missing,Med_D,Follow-Up,75,115,167,80,123.11610188825235,196,21,3,105.35844451055102


### Counting data values for 2 selected columns.

In [8]:
data['Disease_Diagnosed'].value_counts()

Disease_Diagnosed
Flu         20040
Allergy     19987
Cold        19903
Covid-19    19806
missing      5271
Name: count, dtype: int64

In [9]:
data['Average_Heart_Rate'].value_counts()

Average_Heart_Rate
missing    5278
80         3991
78         3953
81         3948
77         3887
           ... 
115          12
116           7
120           5
117           3
119           1
Name: count, Length: 71, dtype: int64

### Replacing values that are 'missing' with NaN and dropping rows with NaN values.

In [10]:
df = data.replace(['missing'], np.nan)
df.dropna(inplace=True)
df

,Patient_Age,Gender,City_of_Residence,State_of_Residence,Has_Insurance,Visited_Last_Month,Payment_Method,Preferred_Doctor,Disease_Diagnosed,Medication_Prescribed,Type_of_Appointment,Average_Heart_Rate,Average_BP,Height_(in_cm),Weight_(in_kg),Payment_Due_($),Last_Visit_(days_ago),Visit_Duration_(mins),Number_of_Tests,Prescription_Cost_($)
0,45,Other,West Brian,Kentucky,Yes,Yes,Insurance,Dr. Williams,Flu,Med_E,General,66,111,183,70,65.45721578126224,193,92,5,15.71751735532889
7,52,Male,Vanessaburgh,Connecticut,Yes,Yes,Insurance,Dr. Williams,Flu,Med_E,General,99,129,170,47,387.08504353508863,179,108,2,76.96592632951703
8,20,Female,West Briannabury,Montana,No,Yes,Card,Dr. Jones,Flu,Med_C,Specialist,67,108,173,86,172.5483427968999,174,91,2,142.31710767890775
11,40,Female,Jonesmouth,Rhode Island,No,No,Card,Dr. Jones,Covid-19,Med_E,Specialist,65,106,166,82,299.1426761400895,193,92,1,77.71536034261702
12,34,Male,West Deborah,Oregon,Yes,Yes,Card,Dr. Jones,Covid-19,Med_B,Specialist,76,111,176,40,13.81128079141103,182,22,5,17.30218505870417
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
104992,44,Female,Port Tracistad,Alaska,Yes,No,Insurance,Dr. Brown,Cold,Med_A,Follow-Up,76,93,178,66,462.56567087397013,189,55,3,59.74631825267156
104993,42,Female,West Annaport,Florida,Yes,No,Cash,Dr. Johnson,Covid-19,Med_D,Specialist,80,129,171,78,131.35632576282202,185,22,6,104.43016736588544
104994,53,Female,Michelleburgh,South Dakota,No,No,Card,Dr. Williams,Flu,Med_B,Emergency,68,109,172,75,488.0316041186948,167,25,5,69.54228578689846
104995,74,Other,West Anthony,Oklahoma,Yes,No,Card,Dr. Williams,Covid-19,Med_E,Specialist,65,112,169,61,309.16444606680665,183,10,3,63.20478776719894


###Check whether 'missing' values are removed by counting values for 2 columns.

In [11]:
df['Disease_Diagnosed'].value_counts()

Disease_Diagnosed
Cold        7598
Flu         7510
Covid-19    7495
Allergy     7463
Name: count, dtype: int64

In [12]:
df['Average_Heart_Rate'].value_counts()

Average_Heart_Rate
80     1212
83     1171
82     1165
77     1163
78     1158
       ... 
116       3
113       3
112       3
119       1
117       1
Name: count, Length: 69, dtype: int64

# Remove any duplicate rows and columns

###Identify duplicate rows

In [13]:
duplicateRows = df[df.duplicated()]
duplicateRows

,Patient_Age,Gender,City_of_Residence,State_of_Residence,Has_Insurance,Visited_Last_Month,Payment_Method,Preferred_Doctor,Disease_Diagnosed,Medication_Prescribed,Type_of_Appointment,Average_Heart_Rate,Average_BP,Height_(in_cm),Weight_(in_kg),Payment_Due_($),Last_Visit_(days_ago),Visit_Duration_(mins),Number_of_Tests,Prescription_Cost_($)
3795,57,Male,Crystaltown,Michigan,No,No,Insurance,Dr. Brown,Flu,Med_A,Specialist,75,94,165,59,343.45383412378794,190,48,3,117.89584180301257
4050,58,Female,Sandrahaven,Wyoming,No,No,Card,Dr. Jones,Allergy,Med_E,General,77,94,167,61,101.19954864899088,185,57,3,121.25842360354149
6602,18,Other,Denisechester,Arkansas,No,No,Cash,Dr. Smith,Covid-19,Med_E,Follow-Up,74,122,184,81,282.2742906140581,174,118,4,147.24684707537554
6619,63,Male,Wesleytown,Kentucky,No,No,Card,Dr. Jones,Allergy,Med_B,Specialist,65,101,163,78,185.30018501132966,170,14,5,49.39500010988799
6636,68,Male,Staceyberg,Arizona,Yes,No,Cash,Dr. Jones,Flu,Med_C,General,76,114,177,75,415.6157371053156,175,91,7,31.0535221126877
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
104888,40,Male,North Ralphhaven,Maine,Yes,No,Cash,Dr. Jones,Covid-19,Med_B,Follow-Up,84,113,175,45,83.18072586624619,172,67,4,94.75701678854188
104889,37,Other,South Steven,Iowa,No,No,Cash,Dr. Williams,Allergy,Med_A,Specialist,87,100,186,58,430.42550174604787,183,44,4,21.25658122239278
104908,43,Other,Rosemouth,Virginia,Yes,Yes,Cash,Dr. Jones,Allergy,Med_E,General,75,93,166,88,149.70457876162595,183,102,6,59.03062699105229
104922,72,Male,Williamsburgh,Vermont,No,No,Card,Dr. Smith,Allergy,Med_D,Emergency,77,130,173,56,237.82102260432842,190,56,4,131.85982773685046


###Dropping duplicated rows

In [14]:
df.drop_duplicates(inplace=True)
df

,Patient_Age,Gender,City_of_Residence,State_of_Residence,Has_Insurance,Visited_Last_Month,Payment_Method,Preferred_Doctor,Disease_Diagnosed,Medication_Prescribed,Type_of_Appointment,Average_Heart_Rate,Average_BP,Height_(in_cm),Weight_(in_kg),Payment_Due_($),Last_Visit_(days_ago),Visit_Duration_(mins),Number_of_Tests,Prescription_Cost_($)
0,45,Other,West Brian,Kentucky,Yes,Yes,Insurance,Dr. Williams,Flu,Med_E,General,66,111,183,70,65.45721578126224,193,92,5,15.71751735532889
7,52,Male,Vanessaburgh,Connecticut,Yes,Yes,Insurance,Dr. Williams,Flu,Med_E,General,99,129,170,47,387.08504353508863,179,108,2,76.96592632951703
8,20,Female,West Briannabury,Montana,No,Yes,Card,Dr. Jones,Flu,Med_C,Specialist,67,108,173,86,172.5483427968999,174,91,2,142.31710767890775
11,40,Female,Jonesmouth,Rhode Island,No,No,Card,Dr. Jones,Covid-19,Med_E,Specialist,65,106,166,82,299.1426761400895,193,92,1,77.71536034261702
12,34,Male,West Deborah,Oregon,Yes,Yes,Card,Dr. Jones,Covid-19,Med_B,Specialist,76,111,176,40,13.81128079141103,182,22,5,17.30218505870417
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
104992,44,Female,Port Tracistad,Alaska,Yes,No,Insurance,Dr. Brown,Cold,Med_A,Follow-Up,76,93,178,66,462.56567087397013,189,55,3,59.74631825267156
104993,42,Female,West Annaport,Florida,Yes,No,Cash,Dr. Johnson,Covid-19,Med_D,Specialist,80,129,171,78,131.35632576282202,185,22,6,104.43016736588544
104994,53,Female,Michelleburgh,South Dakota,No,No,Card,Dr. Williams,Flu,Med_B,Emergency,68,109,172,75,488.0316041186948,167,25,5,69.54228578689846
104995,74,Other,West Anthony,Oklahoma,Yes,No,Card,Dr. Williams,Covid-19,Med_E,Specialist,65,112,169,61,309.16444606680665,183,10,3,63.20478776719894


# **2. Data Transformation**

# Creating new columns based on existing ones

###Converting Average_BP to integer and adding Average_BP_Integer_value column.



In [15]:
df['Average_BP_Integer_value'] = df['Average_BP'].astype(int)
print(df.dtypes)

Patient_Age                 object
Gender                      object
City_of_Residence           object
State_of_Residence          object
Has_Insurance               object
Visited_Last_Month          object
Payment_Method              object
Preferred_Doctor            object
Disease_Diagnosed           object
Medication_Prescribed       object
Type_of_Appointment         object
Average_Heart_Rate          object
Average_BP                  object
Height_(in_cm)              object
Weight_(in_kg)              object
Payment_Due_($)             object
Last_Visit_(days_ago)       object
Visit_Duration_(mins)       object
Number_of_Tests             object
Prescription_Cost_($)       object
Average_BP_Integer_value     int64
dtype: object


###Adding Blood_Pressure_Status column.

In [16]:
threshold_high = 140
df['Blood_Pressure_Status'] = df['Average_BP_Integer_value'].apply(lambda x: 'High' if x >= threshold_high else 'Normal')

In [17]:
print(df[['Patient_Age', 'Average_BP_Integer_value', 'Blood_Pressure_Status']].sample(10))

       Patient_Age  Average_BP_Integer_value Blood_Pressure_Status
7331            40                       109                Normal
66961           32                       113                Normal
83849           58                       123                Normal
64203           47                       134                Normal
104995          74                       112                Normal
6976            58                       129                Normal
2918            41                        96                Normal
59953           35                        90                Normal
90087           70                       116                Normal
67272           22                       124                Normal


###Converting Patient_Age to integer and adding Patient_Age_Integer_Value column.

In [18]:
df['Patient_Age_Integer_Value'] = df['Patient_Age'].astype(int)
print(df.dtypes)

Patient_Age                  object
Gender                       object
City_of_Residence            object
State_of_Residence           object
Has_Insurance                object
Visited_Last_Month           object
Payment_Method               object
Preferred_Doctor             object
Disease_Diagnosed            object
Medication_Prescribed        object
Type_of_Appointment          object
Average_Heart_Rate           object
Average_BP                   object
Height_(in_cm)               object
Weight_(in_kg)               object
Payment_Due_($)              object
Last_Visit_(days_ago)        object
Visit_Duration_(mins)        object
Number_of_Tests              object
Prescription_Cost_($)        object
Average_BP_Integer_value      int64
Blood_Pressure_Status        object
Patient_Age_Integer_Value     int64
dtype: object


###Adding Patient_Birth_Year column.

In [19]:
current_year = 2023
df['Patient_Birth_Year'] = (current_year - df['Patient_Age_Integer_Value'])

In [20]:
print(df[['Patient_Age_Integer_Value', 'Patient_Birth_Year']].sample(10))

       Patient_Age_Integer_Value  Patient_Birth_Year
45373                         24                1999
57235                         39                1984
36480                         23                2000
68404                         36                1987
7173                          38                1985
64260                         55                1968
16155                         43                1980
39641                         42                1981
25756                         60                1963
75405                         71                1952


###Lists all columns.

In [21]:
df.columns

Index(['Patient_Age', 'Gender', 'City_of_Residence', 'State_of_Residence',
       'Has_Insurance', 'Visited_Last_Month', 'Payment_Method',
       'Preferred_Doctor', 'Disease_Diagnosed', 'Medication_Prescribed',
       'Type_of_Appointment', 'Average_Heart_Rate', 'Average_BP',
       'Height_(in_cm)', 'Weight_(in_kg)', 'Payment_Due_($)',
       'Last_Visit_(days_ago)', 'Visit_Duration_(mins)', 'Number_of_Tests',
       'Prescription_Cost_($)', 'Average_BP_Integer_value',
       'Blood_Pressure_Status', 'Patient_Age_Integer_Value',
       'Patient_Birth_Year'],
      dtype='object')

# Aggregating data using groupby and computing summary statistics.

In [22]:
grouped_data = df.groupby('Disease_Diagnosed').agg({'Patient_Age_Integer_Value': 'mean'})
print(grouped_data)

                   Patient_Age_Integer_Value
Disease_Diagnosed                           
Allergy                            44.877345
Cold                               44.695213
Covid-19                           44.471101
Flu                                44.853027


In [23]:
grouped_data2 = df.groupby('Blood_Pressure_Status').agg({'Patient_Age_Integer_Value': 'mean'})
print(grouped_data2)

                       Patient_Age_Integer_Value
Blood_Pressure_Status                           
High                                   45.159624
Normal                                 44.714632


In [24]:
grouped_data3 = df.groupby('Payment_Method').describe()
print(grouped_data3)

               Average_BP_Integer_value                                       
                                  count        mean        std   min    25%   
Payment_Method                                                                
Card                             9546.0  109.595433  14.357340  80.0  100.0  \
Cash                             9647.0  109.735669  14.566804  80.0   99.0   
Insurance                        9473.0  109.376966  14.452292  80.0   99.0   

                                    Patient_Age_Integer_Value             ...   
                  50%    75%    max                     count       mean  ...   
Payment_Method                                                            ...   
Card            110.0  120.0  140.0                    9546.0  44.732558  ...  \
Cash            110.0  120.0  140.0                    9647.0  44.826371  ...   
Insurance       109.0  120.0  140.0                    9473.0  44.612794  ...   

                            Patient_Bi

# Using pivot tables or cross-tabulations for multi-dimensional analysis.

In [25]:
diagnosis_pivot = df.pivot_table(values='Patient_Age_Integer_Value', index='Disease_Diagnosed', columns='Gender', aggfunc='count')
print(diagnosis_pivot)

Gender             Female  Male  Other
Disease_Diagnosed                     
Allergy              2350  2397   2395
Cold                 2396  2417   2415
Covid-19             2367  2289   2455
Flu                  2405  2359   2421


In [26]:
df['Age_Group'] = pd.cut(df['Patient_Age_Integer_Value'], bins=[20, 30, 40, 50, 60, 70, 80, 90], labels=['20-30', '30-40','40-50', '50-60', '60-70', '70-80', '80-90'])
pivot_data = df.pivot_table(index='Age_Group', columns='Gender', values='Average_BP_Integer_value', aggfunc='mean')
print(pivot_data)

Gender         Female        Male       Other
Age_Group                                    
20-30      109.428307  108.837500  109.600000
30-40      110.004623  109.408691  109.482910
40-50      109.796132  109.692087  109.613771
50-60      109.444214  109.686825  109.557393
60-70      109.764398  109.245393  109.816701
70-80      109.241993  110.111455  109.040881
80-90      107.426667  109.790323  107.633803


# **3. Introduction to Alternative Libraries**

# Loading dataset using Polars and Modin.

###Loading dataset with Polars

In [27]:
polars_df = pl.read_csv('/content/healthcare_data_cleaning.csv')
print(polars_df.head())
print(polars_df.tail())

shape: (5, 20)
┌───────────┬────────┬────────────┬────────────┬───┬────────────┬────────────┬────────────┬────────────┐
│ Patient   ┆ Gender ┆ City of    ┆ State of   ┆ … ┆ Last Visit ┆ Visit      ┆ Number of  ┆ Prescripti │
│ Age       ┆ ---    ┆ Residence  ┆ Residence  ┆   ┆ (days ago) ┆ Duration   ┆ Tests      ┆ on Cost    │
│ ---       ┆ str    ┆ ---        ┆ ---        ┆   ┆ ---        ┆ (mins)     ┆ ---        ┆ ($)        │
│ str       ┆        ┆ str        ┆ str        ┆   ┆ str        ┆ ---        ┆ str        ┆ ---        │
│           ┆        ┆            ┆            ┆   ┆            ┆ str        ┆            ┆ str        │
╞═══════════╪════════╪════════════╪════════════╪═══╪════════════╪════════════╪════════════╪════════════╡
│ 45        ┆ Other  ┆ West Brian ┆ Kentucky   ┆ … ┆ 193        ┆ 92         ┆ 5          ┆ 15.7175173 │
│           ┆        ┆            ┆            ┆   ┆            ┆            ┆            ┆ 5532889    │
│ 25        ┆ Female ┆ East Jocel ┆ Colo

###Loading dataset with Modin

In [28]:
modin_df = mpd.read_csv('/content/healthcare_data_cleaning.csv')
print(modin_df.head())
print(modin_df.tail())

  Patient Age  Gender  City of Residence State of Residence Has Insurance   
0          45   Other         West Brian           Kentucky           Yes  \
1          25  Female   East Jocelynfurt           Colorado            No   
2          51   Other  South Lindseyland               Ohio           Yes   
3          18  Female         Taylorfort       Pennsylvania           Yes   
4          48  Female          Birdmouth            Montana            No   

  Visited Last Month Payment Method Preferred Doctor Disease Diagnosed   
0                Yes      Insurance     Dr. Williams               Flu  \
1                Yes           Card      Dr. Johnson               Flu   
2                 No           Cash     Dr. Williams          Covid-19   
3            missing           Cash     Dr. Williams               Flu   
4                 No      Insurance        Dr. Brown               NaN   

  Medication Prescribed Type of Appointment Average Heart Rate Average BP   
0              

# Comparing loading time between Modin and Polars

In [29]:
# Define the dataset path
dataset_path = 'path_to_dataset.gz'

# Using Modin
start_time = time.time()
modin_df = mpd.read_csv('/content/healthcare_data_cleaning.csv')

modin_time = time.time() - start_time

# Using Polars
start_time = time.time()
polars_df = pl.read_csv('/content/healthcare_data_cleaning.csv')

polars_time = time.time() - start_time

# Print execution times

print(f"Modin Execution Time: {modin_time:.4f} seconds")
print(f"Polars Execution Time: {polars_time:.4f} seconds")

Modin Execution Time: 1.9654 seconds
Polars Execution Time: 0.3146 seconds


###Polars requires less time for processing data when compared with Modin. Modin's execution time is 1.9654 seconds, while Polars' execution time is 0.3146 seconds.